In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import numpy as np
%matplotlib inline

In [2]:
df_postulantes_edu_original = pd.read_csv('Originales/fiuba_1_postulantes_educacion.csv')
df_postulantes_datos_original = pd.read_csv('Originales/fiuba_2_postulantes_genero_y_edad.csv')
df_postulantes_vistas_original = pd.read_csv('Originales/fiuba_3_vistas.csv')
df_postulaciones_original = pd.read_csv('Originales/fiuba_4_postulaciones.csv')
df_avisos_online_original = pd.read_csv('Originales/fiuba_5_avisos_online.csv')
df_avisos_detalles_original = pd.read_csv('Originales/fiuba_6_avisos_detalle.csv')

In [3]:
df_postulantes_edu_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_1_postulantes_educacion.csv')
df_postulantes_datos_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_2_postulantes_genero_y_edad.csv')
df_postulantes_vistas_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_3_vistas.csv')
df_postulaciones_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_4_postulaciones.csv')
df_avisos_online_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_5_avisos_online.csv')
df_avisos_detalles_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_6_avisos_detalle.csv')

In [4]:
df_avisos_detalles_faltantes = pd.read_csv('fiuba_6_avisos_detalle_missing_nivel_laboral.csv')

# Detalles de avisos

Veamos el tamaño de los detalles de avisos para generar un dataset para entrenar

In [5]:
df_avisos_detalles_original.shape

(13534, 11)

In [6]:
df_avisos_detalles_nuevo.shape

(18299, 11)

In [7]:
df_avisos_detalles_faltantes.shape

(338, 11)

Juntamos todo

In [8]:
df_avisos_detalles = pd.concat([df_avisos_detalles_original, df_avisos_detalles_nuevo, df_avisos_detalles_faltantes])

In [9]:
df_avisos_detalles.shape

(32171, 11)

Eliminamos duplicados (No tiene sentido que hayan avisos exactamente iguales, es decir, mismo Id de aviso)

In [10]:
df_avisos_detalles = df_avisos_detalles.drop_duplicates()

In [11]:
df_avisos_detalles.shape

(20023, 11)

Veamos las columnas utiles

In [12]:
df_avisos_detalles.columns

Index(['idaviso', 'idpais', 'titulo', 'descripcion', 'nombre_zona', 'ciudad',
       'mapacalle', 'tipo_de_trabajo', 'nivel_laboral', 'nombre_area',
       'denominacion_empresa'],
      dtype='object')

Descartamos las que tengan mayoria de valors nulos, no aportan información relevante

In [13]:
df_avisos_detalles['mapacalle'].isnull().sum()

18349

In [14]:
df_avisos_detalles['ciudad'].isnull().sum()

19921

In [15]:
del df_avisos_detalles['mapacalle']
del df_avisos_detalles['ciudad']

El pais es único y está en todos los datos, no aporta nada

In [16]:
df_avisos_detalles['idpais'].nunique()

1

In [17]:
df_avisos_detalles['idpais'].isnull().sum()

0

In [18]:
del df_avisos_detalles['idpais']

In [19]:
df_avisos_detalles['nombre_zona'].isnull().sum()

0

In [20]:
df_avisos_detalles['nombre_zona'].value_counts()

Gran Buenos Aires              18319
Capital Federal                 1675
Buenos Aires (fuera de GBA)       17
GBA Oeste                         10
Cordoba                            1
Corrientes                         1
Name: nombre_zona, dtype: int64

In [21]:
del df_avisos_detalles['nombre_zona']

Nos quedamos con

In [22]:
df_avisos_detalles.columns

Index(['idaviso', 'titulo', 'descripcion', 'tipo_de_trabajo', 'nivel_laboral',
       'nombre_area', 'denominacion_empresa'],
      dtype='object')

Parseamos el titulo y la descripción para sacar datos relevantes y generar nuevos features

In [23]:
def lowercase_descripcion(row):
    if pd.isnull(row['descripcion']):
        return row['descripcion']   
    
    return row['descripcion'].lower()

def lowercase_titulo(row):
    if pd.isnull(row['titulo']):
        return row['titulo']   
    
    return row['titulo'].lower()

In [24]:
df_avisos_detalles['descripcion'] = df_avisos_detalles.apply(lowercase_descripcion, axis=1)
df_avisos_detalles['titulo'] = df_avisos_detalles.apply(lowercase_titulo, axis=1)

In [25]:
df_avisos_detalles.head()

,idaviso,titulo,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,vendedor/a provincia de santa fe,<p><strong><strong>empresa:</strong></strong> ...,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,enfermeras,<p>solicitamos para importante cadena de farma...,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,chofer de taxi,<p>te gusta manejar? queres ganar plata hacien...,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,chofer de camioneta bahia blanca - punta alta,<p><strong>somos una empresa multinacional que...,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,operarios de planta - rubro electrodomésticos,<p><strong>operarios de planta</strong></p><p>...,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [26]:
def extra(row, list_of_words):
    if pd.isnull(row['descripcion']) and pd.isnull(row['titulo']):
        return 0
    for word in list_of_words:
        if word in row['descripcion'] or word in row['titulo']:
            return 1
    return 0

In [27]:
pal_claves = {
    'web' : ['web', 'pagina', 'página'],
    'crecimiento' : ['crecim', 'progre'],
    'capacitacion' : ['aprender', 'aprendiza', 'capacitar', 'capacitacion', 'capacitación'],
    'clima laboral' : ['excelente cl', 'clima'],
    'multinacional' : ['multin', 'internacional'],
    'incorp_inmediata' : ['incorporación', 'incorporacion'],
    'disponibilidad_horaria' : ['fines de sem', 'sábado', 'sabado', 'domingo', 'feriado', 'rotativo'],
    'disponibilidad_viajar' : ['viaje', 'viajar'],
    'flexibilidad' : ['flexi'],
    'secundario' : ['secundario compl'],
    'terciario' : ['terciario'],
    'ingles' : ['inglés', 'ingles'],
    'estudiantes' : ['universi', 'estudiant'],
    'conoc_informatica' : ['excel', 'word', 'office'],
    'experiencia' : ['años de exp', 'experiencia'],
    'proactivo' : ['proactiv'],
    'relaciones_pers' : ['relaciones', 'personales'],
    'desarrollador' : ['desarr'],
    'cajero' : ['cajero'],
    'vendedor' : ['vended'],
    'ingeniero' : ['ingeni'],
    'analista' : ['analis'],
    'administrativo' : ['administra'],
    'junior' : ['jr', 'junior'],
    'jefe' : ['jefe'],
    'marketing' : ['marke', 'tele'],
    'supervisor' : ['superv']
}

for k, v in pal_claves.items():
    df_avisos_detalles[k] = df_avisos_detalles.apply((lambda x: extra(x, v)), axis=1)

In [28]:
del df_avisos_detalles['descripcion']
del df_avisos_detalles['titulo']

In [29]:
df_avisos_detalles.shape

(20023, 32)

In [30]:
df_avisos_detalles.head()

,idaviso,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,multinacional,...,desarrollador,cajero,vendedor,ingeniero,analista,administrativo,junior,jefe,marketing,supervisor
0,8725750,Full-time,Senior / Semi-Senior,Comercial,VENTOR,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,17903700,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000150677,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000610287,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1000872556,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [31]:
list_tipos_trabajos = []
list_nivel_laboral = []
list_nombre_area = []
list_empresa = []


def tipos_trabajos(row):
    if pd.isnull(row['tipo_de_trabajo']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['tipo_de_trabajo'] in list_tipos_trabajos:
        return list_tipos_trabajos.index(row['tipo_de_trabajo'])+1
    
    list_tipos_trabajos.append(row['tipo_de_trabajo'])
    
    return list_tipos_trabajos.index(row['tipo_de_trabajo'])+1

def nivel_laboral(row):
    if pd.isnull(row['nivel_laboral']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['nivel_laboral'] in list_nivel_laboral:
        return list_nivel_laboral.index(row['nivel_laboral'])+1
    
    list_nivel_laboral.append(row['nivel_laboral'])
    
    return list_nivel_laboral.index(row['nivel_laboral'])+1

def nombre_area(row):
    if pd.isnull(row['nombre_area']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['nombre_area'] in list_nombre_area:
        return list_nombre_area.index(row['nombre_area'])+1
    
    list_nombre_area.append(row['nombre_area'])
    
    return list_nombre_area.index(row['nombre_area'])+1

def nombre_empresa(row):
    if pd.isnull(row['denominacion_empresa']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['denominacion_empresa'] in list_empresa:
        return list_empresa.index(row['denominacion_empresa'])+1
    
    list_empresa.append(row['denominacion_empresa'])
    
    return list_empresa.index(row['denominacion_empresa'])+1

df_avisos_detalles['tipo_de_trabajo'] = df_avisos_detalles.apply(tipos_trabajos, axis=1)
df_avisos_detalles['nivel_laboral'] = df_avisos_detalles.apply(nivel_laboral, axis=1)
df_avisos_detalles['nombre_area'] = df_avisos_detalles.apply(nombre_area, axis=1)
df_avisos_detalles['denominacion_empresa'] = df_avisos_detalles.apply(nombre_empresa, axis=1)

In [32]:
df_avisos_detalles.head()

,idaviso,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,multinacional,...,desarrollador,cajero,vendedor,ingeniero,analista,administrativo,junior,jefe,marketing,supervisor
0,8725750,1,1,1,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,17903700,1,1,2,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1000150677,1,1,3,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1000610287,1,1,3,4,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1000872556,1,1,4,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [33]:
df_avisos_detalles.shape

(20023, 32)

In [34]:
df_avisos_online = pd.concat([df_avisos_online_original, df_avisos_online_nuevo])
df_avisos_online = df_avisos_online.drop_duplicates()

In [35]:
df_avisos_online['online'] = 1
df_avisos_detalles = pd.merge(df_avisos_detalles, df_avisos_online, how='left', on=['idaviso'])

In [36]:
df_avisos_detalles['online'] = df_avisos_detalles['online'].fillna(value=0)

In [37]:
df_avisos_detalles.shape

(20023, 33)

In [61]:
df_avisos_detalles.to_csv('Predecir/avisos_detalles.csv')

# Postulantes

In [38]:
df_postulantes_edu = pd.concat([df_postulantes_edu_original, df_postulantes_edu_nuevo])

In [39]:
df_postulantes_edu.shape

(705684, 3)

In [40]:
df_postulantes_edu.columns

Index(['idpostulante', 'nombre', 'estado'], dtype='object')

In [41]:
df_postulantes_edu.head()

,idpostulante,nombre,estado
0,NdJl,Posgrado,En Curso
1,8BkL,Universitario,En Curso
2,1d2B,Universitario,En Curso
3,NPBx,Universitario,En Curso
4,NPBx,Master,En Curso


In [42]:
df_postulantes_edu['nombre'].value_counts()

Universitario        259550
Secundario           246012
Terciario/Técnico    114815
Otro                  53513
Posgrado              20885
Master                10297
Doctorado               612
Name: nombre, dtype: int64

In [43]:
nivel_dict = {'Universitario' : 3
             , 'Secundario' : 0
             , 'Terciario/Técnico' : 1
             , 'Otro': 2
             , 'Posgrado' : 5
             , 'Master' : 6
             , 'Doctorado' : 7}

def educacion(row):
    return nivel_dict[row['nombre']]

df_postulantes_edu['nombre'] = df_postulantes_edu.apply(educacion, axis=1)

In [44]:
df_postulantes_edu['estado'].value_counts()

Graduado      460482
En Curso      184467
Abandonado     60735
Name: estado, dtype: int64

In [45]:
estado_dict = {'Graduado' : 2
             , 'En Curso' : 1
             , 'Abandonado' : 0}

def estado(row):
    return estado_dict[row['estado']]

df_postulantes_edu['estado'] = df_postulantes_edu.apply(estado, axis=1)

In [46]:
df_postulantes_edu.shape

(705684, 3)

Nos quedamos unicamente con su mayor nivel de estudio

In [47]:
for_join = df_postulantes_edu.loc[:, ['idpostulante', 'nombre']].groupby('idpostulante').max().reset_index()
df_postulantes_edu = pd.merge(df_postulantes_edu, for_join, how='inner', on=['idpostulante', 'nombre'])

In [48]:
df_postulantes_edu.shape

(477848, 3)

In [49]:
df_postulantes_edu.head()

,idpostulante,nombre,estado
0,NdJl,5,1
1,NdJl,5,1
2,1d2B,3,1
3,NPBx,6,1
4,NPBx,6,1


Eliminamos duplicados

In [50]:
df_postulantes_edu = df_postulantes_edu.drop_duplicates()

In [51]:
df_postulantes_edu.shape

(388287, 3)

In [52]:
df_postulantes_datos = pd.concat([df_postulantes_datos_original, df_postulantes_datos_nuevo])
df_postulantes_datos = df_postulantes_datos.drop_duplicates()

In [53]:
df_postulantes_datos.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [54]:
df_postulantes_datos['edad'] = pd.to_datetime(df_postulantes_datos["fechanacimiento"], errors = 'coerce')
del df_postulantes_datos['fechanacimiento']

In [55]:
list_sexos = []

def tipo_sexos(row):
    if pd.isnull(row['sexo']): #Es nula, seteo valor 0 por defecto
        return 0
    if row['sexo'] in list_sexos:
        return list_sexos.index(row['sexo'])+1
    
    list_sexos.append(row['sexo'])
    
    return list_sexos.index(row['sexo'])+1

df_postulantes_datos['sexo'] = df_postulantes_datos.apply(tipo_sexos, axis=1)

In [56]:
def edad(row):
    return 2018-row['edad'].year

df_postulantes_datos['edad'] = df_postulantes_datos.apply(edad, axis=1)

In [57]:
df_postulantes_datos.shape

(408452, 3)

Elimino outliers

In [58]:
df_postulantes_datos = df_postulantes_datos.loc[(df_postulantes_datos['edad']>18) & (df_postulantes_datos['edad']<70), :]

In [59]:
df_postulantes_datos.shape

(386727, 3)

In [66]:
df_postulantes = pd.merge(df_postulantes_edu, df_postulantes_datos, how='inner', on=['idpostulante'])

In [67]:
df_postulantes.to_csv('Predecir/postulantes_datos.csv')

# Postulaciones con postulantes (To train)

In [70]:
df_postulaciones = pd.concat([df_postulaciones_nuevo, df_postulaciones_original])

In [71]:
df_postulaciones.shape

(8311264, 3)

In [72]:
df_postulaciones.columns

Index(['idaviso', 'idpostulante', 'fechapostulacion'], dtype='object')

Elimino duplicados

In [73]:
df_postulaciones = df_postulaciones.drop_duplicates()

In [74]:
df_postulaciones.shape

(6604534, 3)

In [75]:
df_postulaciones['fechapostulacion'] = pd.to_datetime(df_postulaciones["fechapostulacion"], errors = 'coerce')

In [79]:
df_postulaciones['fechapostulacion'].min()

Timestamp('2018-01-15 00:00:01')

In [78]:
df_postulaciones['fechapostulacion'].max()

Timestamp('2018-04-17 10:42:39')

In [82]:
def get_hour(row):
    return row['fechapostulacion'].hour

def get_day(row):
    return row['fechapostulacion'].day

def get_month(row):
    return row['fechapostulacion'].month


df_postulaciones['hora'] = df_postulaciones.apply(get_hour, axis=1)
df_postulaciones['dia'] = df_postulaciones.apply(get_day, axis=1)
df_postulaciones['mes'] = df_postulaciones.apply(get_month, axis=1)

In [84]:
del df_postulaciones['fechapostulacion']

In [85]:
df_postulaciones.head()

,idaviso,idpostulante,hora,dia,mes
0,1112248724,NjlD,7,19,1
1,1112286523,ZaO5,15,24,1
2,1112272060,ZaO5,15,24,1
3,1112288401,ZaO5,8,26,1
4,1112300563,ZaO5,13,30,1


In [86]:
df_postulaciones.to_csv('df_postulaciones.csv')

In [87]:
df_postulaciones.shape

(6604534, 5)

In [88]:
del df_postulaciones['hora']
del df_postulaciones['dia']
del df_postulaciones['mes']

In [89]:
df_postulaciones.to_csv('Predecir/df_postulaciones.csv')